<a href="https://colab.research.google.com/github/perrin-isir/tp_classif_images/blob/main/tp_classif_images.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>
<a id="raw-url" href="https://raw.githubusercontent.com/perrin-isir/tp_classif_images/main/tp_classif_images.ipynb" download> <img align="left" src="https://img.shields.io/badge/Github-Download%20(Right%20click%20%2B%20Save%20link%20as...)-blue" alt="Download (Right click + Save link as)" title="Download Notebook"></a>

**IMPORTANT :** Ce notebook fonctionne mieux avec accélération GPU.  
Dans Colab : dans le menu Colab, choisissez Runtime > Change Runtime Type, puis sélectionnez 'GPU'.

In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torchvision.utils as vutils
from PIL import ImageFile
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

learning_rate = 0.01
momentum = 0.5
batch_size_train = 40
batch_size_test = 500

In [ ]:
tmp_dir = os.path.join(os.path.expanduser("~"), "tmp_data")
# save dataset zip file in /root/tmp_data/, and unzip data
if not os.path.isfile(os.path.join(os.path.expanduser("~"), "tmp_data", "dataset.zip")):
    !mkdir {os.path.join(os.path.expanduser("~"), "tmp_data")}
    !wget -P {tmp_dir} -O {os.path.join(os.path.expanduser("~"), "tmp_data", "dataset.zip")} "https://filesender.renater.fr/download.php?token=ee514cca-c74a-48ac-858d-c465ce643d01&files_ids=33530618"
    !unzip --qq -d {os.path.join(os.path.expanduser("~"), "tmp_data")} {os.path.join(os.path.expanduser("~"), "tmp_data", "dataset.zip")}

In [ ]:
# dataloader class and function

ImageFile.LOAD_TRUNCATED_IMAGES = True


class Data:
    def __init__(self, dataset_train, dataset_train_original, dataloader_train,
                 dataset_test, dataset_test_original, dataloader_test,
                 batch_size_train, batch_size_test):
        self.train = dataset_train
        self.train_original = dataset_train_original
        self.loader_train = dataloader_train
        self.num_train_samples = len(dataset_train)
        self.test = dataset_test
        self.test_original = dataset_test_original
        self.loader_test = dataloader_test
        self.num_test_samples = len(dataset_test)
        self.batch_size_train = batch_size_train
        self.batch_size_test = batch_size_test


def loadImgs(des_dir, img_size=100, batch_size_train=40, batch_size_test=100):

    dataset_train = dset.ImageFolder(root=os.path.join(des_dir, "train"),
                               transform=transforms.Compose([
                                   transforms.Resize(img_size),
                                   transforms.RandomCrop(75, padding=4),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Lambda(lambda x: x.to(device)),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                               ]))

    dataset_train_original = dset.ImageFolder(root=os.path.join(des_dir, "train"),
                               transform=transforms.Compose([
                                   transforms.Resize(img_size),
                                   transforms.ToTensor(),
                                   transforms.Lambda(lambda x: x.to(device)),
                               ]))

    dataset_test = dset.ImageFolder(root=os.path.join(des_dir, "test"),
                               transform=transforms.Compose([
                                   transforms.Resize(img_size),
                                   transforms.RandomCrop(75, padding=4),
                                   transforms.RandomHorizontalFlip(),
                                   transforms.ToTensor(),
                                   transforms.Lambda(lambda x: x.to(device)),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                               ]))

    dataset_test_original = dset.ImageFolder(root=os.path.join(des_dir, "test"),
                               transform=transforms.Compose([
                                   transforms.Resize(img_size),
                                   transforms.ToTensor(),
                                   transforms.Lambda(lambda x: x.to(device)),
                               ]))

    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size_train, shuffle=True)

    dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size_test, shuffle=True)

    data = Data(dataset_train, dataset_train_original, dataloader_train,
                dataset_test, dataset_test_original, dataloader_test,
                batch_size_train, batch_size_test)
    return data

In [ ]:
# plotting tool

def plotdata(data, indexes, model=None, original=True):
    l = []
    l_original = []
    for elt in indexes:
        t, _ = data.test[elt]
        l.append(t)
        tbis, _ = data.test_original[elt]
        l_original.append(tbis)
    cte = 30.0/9.0
    k = len(l)
    n = int(np.sqrt(cte * k))
    m = int(k/(n * 1.0))
    if (n*m<k):
        m = m+1
    width=20
    mult = 2
    if model is None:
        mult = 1
    fig, ax = plt.subplots(mult*m,n,squeeze=False, figsize=(width,int(width*m/(n*2.0))))
    for i in range(m):
        if model is not None:
            for j in range(n):
                if(j+n*i < k):
                    ax[mult*i+1,j].tick_params(axis=u'both', which=u'both',length=0)
                    ax[mult*i+1,j].set_ylim([-0.5,10.5])
                    ax[mult*i+1,j].set_xlim([-1.5,12.5])
                    ax[mult*i+1,j].set_xticks([])
                    ax[mult*i+1,j].set_xticks(np.arange(0.5,9.5,1), minor=True)
                    ax[mult*i+1,j].set_yticks([])
                    ax[mult*i+1,j].grid(False)
                    ax[mult*i+1,j].set_aspect('equal')
                    L = F.softmax(model(l[j+n*i]), dim=1).cpu().data.numpy().flatten()
                    C = [(0.9, 0.1, 0.0, 1.0), (0.0, 0.1, 0.9, 1.0)]
                    ax[mult*i+1,j].barh([1,5], [z * 10.0 for z in reversed(L)], color=C)
                    for idx in range(len(L)):
                        if L[idx]>0.02:
                            ax[mult*i+1,j].text(10.0*L[idx]+0.15,(len(L)-1-idx+0.1)*4.0,idx)
                else:
                    ax[mult*i+1,j].axis('off')
        # ------------------
        for j in range(n):
            if(j+n*i < k):
                ax[mult*i+0,j].tick_params(axis=u'both', which=u'both',length=0)
                ax[mult*i+0,j].set_xticks([])
                ax[mult*i+0,j].set_yticks([])
                ax[mult*i+0,j].grid(False)
                ax[mult*i+0,j].set_xticklabels([])
                ax[mult*i+0,j].set_yticklabels([])
                img = None
                if original:
                    img = l_original[j + n * i]
                else:
                    img = l[j + n * i]
                if img.shape[0] == 3:
                    N = img[:, :, :]
                    ax[mult * i + 0, j].imshow(N.permute(1, 2, 0).cpu().data.numpy(), )
                else:
                    N = img[0, :, :]
                    ax[mult*i+0,j].matshow(N.cpu().data.numpy(), cmap='Greys', )
            else:
                ax[mult*i+0,j].axis('off')
    return fig, ax

In [ ]:
# evaluation on a batch of test data:

def evaluate(model, data):
    batch_enum = enumerate(data.loader_test)
    batch_idx, (testdata, testtargets) = next(batch_enum)
    testdata = testdata.to(device)
    testtargets = testtargets.to(device)
    model = model.eval()
    oupt = torch.argmax(model(testdata), dim=1)
    t = torch.sum(oupt == testtargets)
    result = t * 100.0 / len(testtargets)
    model = model.train()
    print(f"{t} correct on {len(testtargets)} ({result.item()} %)")
    return result.item()

In [ ]:
# iterative training on batches for one epoch:

def train_epoch(model, optimizer, data):
    batch_enum = enumerate(data.loader_train)
    i_count = 0
    iterations = data.num_train_samples // data.batch_size_train
    for batch_idx, (dt, targets) in batch_enum:
        i_count = i_count+1
        outputs = model(dt.to(device))
        loss = F.cross_entropy(outputs, targets.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if not i_count % 30:
            print(f"    batch {i_count} / {iterations}")
        if i_count == iterations:
            break

In [ ]:
# define the neural net

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.25)
        self.dropout3 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(((((75-2)//2-2)//2)**2)*64, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x.view(-1, 3, 75, 75)))
        x = self.dropout1(F.max_pool2d(x, 2))
        x = F.relu(self.conv2(x))
        x = self.dropout2(F.max_pool2d(x, 2))
        x = torch.flatten(x, 1)
        x = self.dropout3(F.relu(self.fc1(x)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# load the data

data = loadImgs(os.path.join(os.path.expanduser("~"), "tmp_data"), batch_size_train=batch_size_train, batch_size_test=batch_size_test)

In [ ]:
# initialize the model

net = Net().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
# net.load_state_dict(torch.load(os.path.join(os.path.expanduser("~"), "tmp_data", "model_TP.pt")))


In [ ]:
# evaluate the model on one batch of test data (expecting ~50% accuracy after initialization)
evaluate(net, data)

In [ ]:
# plot results on random images from the test set

indices = np.random.choice(range(data.num_test_samples), 10)
fig, ax = plotdata(data, indices, net, original=True)

In [ ]:
# train for one epoch, and save the model
num_epochs = 1
for j in range(num_epochs):
    print(f"epoch {j+1} / {num_epochs}")
    train_epoch(net, optimizer, data)
    evaluate(net, data)
    torch.save(net.state_dict(), os.path.join(os.path.expanduser("~"), "tmp_data", "model_TP.pt"))